## `SciPy.fftpack` - Rychlá Fourierova transformace
`scipy.fftpack` je modul, který poskytuje nástroje pro rychlé Fourierovy transformace (FFT) a jejich inverzní operace. FFT je algoritmus, který umožňuje rychle vypočítat diskrétní Fourierovu transformaci (DFT) a její inverzní. DFT je matematický nástroj používaný k analýze a zpracování signálů, zejména pro rozklad signálů na frekvenční spektrum.

In [ ]:
import numpy as np
from scipy import fftpack
from IPython.display import Audio
import matplotlib.pyplot as plt


In [ ]:

# Nastavení vzorkovací frekvence pro zvukový signál
sample_rate = 8000
# délka signálu
duration = 1

# Vytvoříme časovou osu
t = np.linspace(0, duration, duration * sample_rate)

# Definujeme základní frekvenci a harmonické
freq = 800
harmonic2 = 2 * freq
harmonic3 = 3 * freq

# Vytvoříme signál s harmonickými frekvencemi
signal = (np.sin(2 * np.pi * freq * t) +
    0.5 * np.sin(2 * np.pi * harmonic2 * t) +
    0.3 * np.sin(2 * np.pi * harmonic3 * t))

# Přidáme šum do signálu
noisy_signal = signal + 0.5 * np.random.randn(len(signal))

In [ ]:
# čistý signál


# Use IPython.display.Audio to play back the signal
Audio(data=signal, rate=sample_rate)

In [ ]:
# zašuměný signál
# přehrajeme signál
Audio(data=noisy_signal, rate=sample_rate)

In [ ]:
# Aplikujeme rychlou Fourierovu transformaci (FFT)
noisy_fft = fftpack.fft(noisy_signal)
clean_fft = fftpack.fft(signal)

# Vypočítáme frekvenční spektrum
frequencies = fftpack.fftfreq(len(signal), duration/sample_rate)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(t, signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(clean_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')
ax2.set_xlim(0, 2500)

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(t, noisy_signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(noisy_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')
ax2.set_xlim(0, 2500)

plt.tight_layout()
plt.show()


In [ ]:
# vidíme, že šum je v frekvenčním spektru pěkně oddělen od základní frekvence

# Filtrujeme šum v frekvenčním spektru, tak, že vypustíme všechny malé amplitudy
mask = np.abs(noisy_fft) > 200
filtered_fft = noisy_fft * mask

# Aplikujeme inverzní rychlou Fourierovu transformaci (IFFT)
filtered_signal = np.real(fftpack.ifft(filtered_fft))


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax1.plot(t, filtered_signal)
ax1.set_title('Časový průběh signálu')
ax1.set_xlabel('Čas [s]')
ax1.set_ylabel('Amplituda')
ax1.set_xlim(0, 0.01)

ax2.plot(frequencies, np.abs(filtered_fft))
ax2.set_title('Frekvenční spektrum')
ax2.set_xlabel('Frekvence [Hz]')
ax2.set_ylabel('Amplituda')
ax2.set_xlim(0, 2500)

plt.tight_layout()
plt.show()


In [ ]:
# filtrovaný signál
# přehrajeme signál
Audio(data=filtered_signal, rate=sample_rate)
